# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   


In [1]:
%pip install pennylane > o

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp
from scipy.special import lpmv
from scipy.special import sph_harm


In [3]:
nCubits = 4
theta = math.pi/5
phi = math.pi/7

In [151]:
dev = qml.device ("default.qubit", wires = nCubits)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])

def ProductosPotencias(theta, nCubits):
  """
  Obtener una matriz con la evaluación de los términos cos^k(theta)·sen^l(theta) con l+k = n.
  
  Parámetros:
    theta: el ángulo a aplicar a las funciones trigonométricas.
    nCubits: cantida de cúbits a utilizar, esto es el n = k+l.

  Retorno:
    Matriz con los terminos cos^k(theta)·sen^l(theta) con l+k = n.
  """
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  cos, sen = sp.symbols("cos sen")

def ExtraerTerminosBinomio(probsArray, nCubits = 3):
  """
  Extrae el valor los terminos de la expansión de la potencia de binomios (cos{theta}+bsen{theta})^n a partir de la matriz de probalidades obtenida de un circuito cuántico que aplica la misma rotación a toodos los cúbits.

  Parámetros:
    probArray: el array de probabilidades.
    nCubits: cantidad de cúbits del circuito del que se ha obtenido e array
  
  Retorno:
    Array de tuplas en las que el primer término es un simbolo de sympy con la expresión del témino binomial y el segundo es el valor.
  """
  cos, sen = sp.symbols("cos sen")
  res = []
  for i in range(nCubits +1):
    potenciaCoseno = cos**((nCubits - i)* 2) #como es un valor de problabilidad se multiplica por 2 cada potencia
    potenciaSeno = sen**(i * 2)
    res.append((potenciaCoseno * potenciaSeno, probsArray[2**(i) -1]))
  print(res)
  return res

def PotenciasTrig(terminosBinomio, nCubits = 3, funcion= sp.symbols("cos")):
  """
  Obener el valor del coseno a la 'n' potencia para el ángulo 'theta'. El resultado es una lista con todas las potencias del coseno de 0 a nCubits
  """
  cos, sen = sp.symbols("cos sen")
  c = cos**2
  s = sen**2
  arr = []
  for i in range(nCubits):  #Expansión de los distintos terminos (c+s)^k(c-s)^l con k+l = nCubits.
    potenciaSuma = (c + s)**(nCubits-i - 1)
    potenciaResta = (c - s)**(i + 1)
    productoPotencias = potenciaSuma * potenciaResta
    arr.append(productoPotencias.expand())
  res = []
  for potencia, i in zip(arr, range(nCubits)): #Sustitución de los valores obtenidos del cicuito cuántico en la expansión calculada
    for termino in terminosBinomio:
      potencia = potencia.subs(termino[0], termino[1])
    res.append((funcion**(i+1), potencia))
  return res

def EvalLegendre(theta, ncubits = 4):
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  PolinomiosLegendre = {'P_0_0': 1}
  PolinomiosLegendre['P_1_0']= C[0] + 2*C[1] -2*C[7] -C[15]
  PolinomiosLegendre['P_1_1']= -S[0] - 2*S[1] + 2*S[7] + S[15]
  PolinomiosLegendre['P_2_0']= (3*(C[0] - 2*C[3] + C[15]) - 1)/2
  PolinomiosLegendre['P_2_1']= 3*(C[0] + 2*C[1] - 2*C[7] - C[15])*(-S[0] -2*S[1] + 2*S[7] + S[15])
  PolinomiosLegendre['P_2_2']= 3*(S[0] - 2*S[3] + S[15])
  PolinomiosLegendre['P_3_0']= C[0] - 8*C[1] + 8*C[7] - C[15]
  PolinomiosLegendre['P_3_1']= -(3/2)*(-S[0] + 18*S[1] -18*S[7] + S[15])
  PolinomiosLegendre['P_3_2']= 15*(4*C[1] - 4*C[7])
  PolinomiosLegendre['P_3_3']= -15*(S[0] -2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_0']= (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[15] + 3)
  PolinomiosLegendre['P_4_1']= -(5/2)*(7*(C[0] +2*C[1] -2*C[7] - C[15])*(4*S[1]-4*S[7]) + 3*(C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15]))
  PolinomiosLegendre['P_4_2'] = (15/2)*(56*C[1] - 112*C[3] + 56*C[7] - S[0] - 28*S[1] + 58*S[3] - 28*S[7] - S[15])
  PolinomiosLegendre['P_4_3'] = -105*(C[0] + 2*C[1] - 2*C[7] - C[15])*(S[0] - 2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_4'] = 105*(S[0] - 4*S[1] + 6*S[3] - 4*S[7] + S[15])
  return PolinomiosLegendre

def EvalEsfericos(theta, phi, l = 4):
  legendre = EvalLegendre(theta)
  AE = {}
  for k in range(l+1):
    for m in range(-k, k+1):
      condonShortley = ((-1)**abs(m))
      coef = condonShortley * math.sqrt(((2*k + 1)*math.factorial(k - abs(m))) / (4*math.pi * math.factorial(k+abs(m)))) #Coeficiente de normalización
      e = ExpPhi(phi, (m))
      legendre_normalizado = (coef * legendre['P_' + str(k) + '_' + str(abs(m))])
      AE['Y_' + str(k) + '_' + str(m)] = ( legendre_normalizado * e[0], legendre_normalizado * e[1])
  
  return AE

In [5]:
#c = cos^2(theta/2)
#s = sen^2(theta/2)
cos, sen = sp.symbols("cos sen")
#c, s = sp.symbols("c s")
c = cos**2
s = sen**2
arr = []
nCubits = 4
for i in range(nCubits):
  potenciaSuma = (c + s)**(nCubits-i - 1)
  potenciaResta = (c - s)**(i + 1)
  productoPotencias = potenciaSuma * potenciaResta
  a = productoPotencias.expand()
  print(a)
  arr.append(a)
for x in arr:
  print(x.subs(cos**6*sen**2, 1))

cos**8 + 2*cos**6*sen**2 - 2*cos**2*sen**6 - sen**8
cos**8 - 2*cos**4*sen**4 + sen**8
cos**8 - 2*cos**6*sen**2 + 2*cos**2*sen**6 - sen**8
cos**8 - 4*cos**6*sen**2 + 6*cos**4*sen**4 - 4*cos**2*sen**6 + sen**8
cos**8 - 2*cos**2*sen**6 - sen**8 + 2
cos**8 - 2*cos**4*sen**4 + sen**8
cos**8 + 2*cos**2*sen**6 - sen**8 - 2
cos**8 + 6*cos**4*sen**4 - 4*cos**2*sen**6 + sen**8 - 4


In [6]:
((-1)**1)*math.sqrt((((2*1 + 1)*math.factorial(1 - abs(1))) / (4*math.pi * math.factorial(1+abs(1)))))
math.sqrt(3/(8*math.pi))

0.3454941494713355

In [152]:
AE = EvalEsfericos(theta, phi)
for armonicos in AE.values():
    print(armonicos[0], armonicos[1])

0.28209479177387814 0.0
0.18296548340389993 -0.0881115329251683
0.3952877356237497 0.0
0.18296548340389993 0.08811153292516832
0.08320749142747604 -0.10433889404646438
0.3309876688616823 -0.15939526046205713
0.3038878129445756 0.0
0.3309876688616823 0.15939526046205718
0.08320749142747604 0.10433889404646438
0.018853675536764813 -0.0826033497217849
0.17810212184279137 -0.22333299684443067
0.38894232429676284 -0.18730475156132773
0.08228221038686168 0.0
0.38894232429676284 0.18730475156132778
0.17810212184279137 0.22333299684443067
0.018853675536764813 0.0826033497217849
-0.011754143143792865 -0.051498265942538306
0.045758831747021826 -0.2004825411516632
0.25808646471523716 -0.3236301904404923
0.3205626929630728 -0.15437485666760664
-0.1736900156399915 -0.0
0.3205626929630728 0.15437485666760667
0.25808646471523716 0.3236301904404923
0.045758831747021826 0.2004825411516632
-0.011754143143792865 0.0514982659425383


In [117]:
m = [0,-1,0,1,-2, 1,0,1,2,-3,-2,-1,0,1,2,3,-4,-3,-2,-1,0,1,2,3,4]
v = [0, 1,1,1, 2, 2,2,2,2, 3, 3, 3,3,3,3,3, 4, 4, 4, 4,4,4,4,4,4]
x = []
for i in range(15):
    x.append(math.cos(theta))

sph = sph_harm(m, v, phi, theta)
sph

array([ 0.28209479+0.j        ,  0.18296548-0.08811153j,
        0.39528774+0.j        , -0.18296548-0.08811153j,
        0.08320749-0.10433889j, -0.33098767-0.15939526j,
        0.30388781+0.j        , -0.33098767-0.15939526j,
        0.08320749+0.10433889j,  0.01885368-0.08260335j,
        0.17810212-0.223333j  ,  0.38894232-0.18730475j,
        0.08228221+0.j        , -0.38894232-0.18730475j,
        0.17810212+0.223333j  , -0.01885368-0.08260335j,
       -0.01175414-0.05149827j,  0.04575883-0.20048254j,
        0.25808646-0.32363019j,  0.32056269-0.15437486j,
       -0.17369002+0.j        , -0.32056269-0.15437486j,
        0.25808646+0.32363019j, -0.04575883-0.20048254j,
       -0.01175414+0.05149827j])

In [18]:
for k in range(4+1):
    for m in range(-k, k+1):
        c = ((-1)**m)*math.sqrt((((2*k + 1)*math.factorial(k - abs(m))) / (4*math.pi * math.factorial(k+abs(m)))))
        print(c)

0.28209479177387814
-0.3454941494713355
0.4886025119029199
-0.3454941494713355
0.12875806734106318
-0.25751613468212636
0.6307831305050401
-0.25751613468212636
0.12875806734106318
-0.027814921575518937
0.06813236509555216
-0.21545345607610045
0.7463526651802308
-0.21545345607610045
0.06813236509555216
-0.027814921575518937
0.004214597070904597
-0.011920680675222404
0.044603102903819275
-0.18923493915151202
0.8462843753216345
-0.18923493915151202
0.044603102903819275
-0.011920680675222404
0.004214597070904597


In [10]:
4*35

140

In [11]:
#math.sin(theta)**1
#math.cos(theta/2)**8 + 2*math.cos(theta/2)**6*math.sin(theta/2)**2 - 2*math.cos(theta/2)**2*math.sin(theta/2)**6  - math.sin(theta/2)**8
#A = [[a, -b],[b, 1]]
print(abs(-3))

3


In [12]:
cos = sp.symbols("cos")
sen = sp.symbols("sin")
cos2 = sp.symbols("cos2")
sen2 = sp.symbols("sin2")

C0 = cos**8
C1 = cos**6*sen**2
C3 = cos**4*sen**4
C7 = cos**2*sen**6
C15 = sen**8

S0 = cos2**8
S1 = cos2**6*sen2**2
S3 = cos2**4*sen2**4
S7 = cos2**2*sen2**6
S15 = sen2**8

X = (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[7] + 3)
X = (35*(C[0] - 4*C[1] + 6*C[3] - 4*C[7] + C[15]) - 30*(C[0] -2*C[3]  + C[15]) + 3)/8
X = (C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15])

X


NameError: name 'C' is not defined

In [70]:
 legendre = EvalLegendre(theta)
legendre

{'P_0_0': 1,
 'P_1_0': tensor(0.80901699, requires_grad=True),
 'P_1_1': tensor(-0.58778525, requires_grad=True),
 'P_2_0': tensor(0.48176275, requires_grad=True),
 'P_2_1': tensor(-1.42658477, requires_grad=True),
 'P_2_2': tensor(1.03647451, requires_grad=True),
 'P_3_0': tensor(0.11024575, requires_grad=True),
 'P_3_1': tensor(-2.00365044, requires_grad=True),
 'P_3_2': tensor(4.19262746, requires_grad=True),
 'P_3_3': tensor(-3.04612215, requires_grad=True),
 'P_4_0': tensor(-0.20523836, requires_grad=True),
 'P_4_1': tensor(-1.88019056, requires_grad=True),
 'P_4_2': tensor(9.28048775, requires_grad=True),
 'P_4_3': tensor(-17.25055212, requires_grad=True),
 'P_4_4': tensor(12.53325974, requires_grad=True)}

In [75]:
m = [0,0,1,0,1,2,0,1,2,3,0,1,2,3,4]
v = [0,1,1,2,2,2,3,3,3,3,4,4,4,4,4]
lp = lpmv(m, v, math.cos(theta))
lp

array([  1.        ,   0.80901699,  -0.58778525,   0.48176275,
        -1.42658477,   1.03647451,   0.11024575,  -2.00365044,
         4.19262746,  -3.04612215,  -0.20523836,  -1.88019056,
         9.28048775, -17.25055212,  12.53325974])

In [89]:
m, k= sp.symbols('m, k')
#x = ((-1)**sp.Abs(m))*math.sqrt(((2*k + 1)*sp.factorial(k - sp.Abs(m))) / (4*sp.pi * sp.factorial(k+sp.Abs(m))))
x = ((-1)**sp.Abs(m))*sp.sqrt(((2*k + 1) * sp.factorial(k - sp.Abs(m))) / (4*sp.pi * sp.factorial(k + sp.Abs(m))))
x

(-1)**Abs(m)*sqrt((2*k + 1)*factorial(k - Abs(m))/factorial(k + Abs(m)))/(2*sqrt(pi))